In [85]:
from PIL import Image as Image_main
from PIL.Image import Image
import cv2
import numpy


def rotate(image, angle, scale = 1.0):
    '''
    Return rotated image
    '''
    (h, w) = image.shape[:2]
    if center is None:
        center = (w / 2, h / 2)
    # Rotation matrix
    M = cv2.getRotationMatrix2D(center, angle, scale)
    rotated = cv2.warpAffine(image, M, (w, h))
    return rotated


def get_rect_list(contours):
    '''
    filter to get a list of rectangles contours
    '''
    rectangle_contours = []
    for contour in contours:
        perimeter = cv2.arcLength(contour, True)
        approximationAccuracy = 0.02 * perimeter
        approximation = cv2.approxPolyDP(contour, approximationAccuracy, True)
        if len(approximation) == 4:
            rectangle_contours.append(contour)
    return rectangle_contours
    

## Car image
image_path = './images/car2.jpg'
image_PIL = Image_main.open(image_path)
image_CV = cv2.cvtColor(numpy.array(image_PIL), cv2.COLOR_RGB2BGR)

## Plate image
plate_path = './Kifal_plate.png'
plate_PIL = Image_main.open(plate_path)
plate_CV = cv2.cvtColor(numpy.array(plate_PIL), cv2.COLOR_RGB2BGR)

# Show Original image:
#cv2.imshow('Original Image', image_CV)

### Preprocessing the image
gray = cv2.cvtColor(image_CV, cv2.COLOR_BGR2GRAY)
#cv2.imshow('Gray Scaled', gray)
bilateral = cv2.bilateralFilter(gray, 11, 17, 17)
#cv2.imshow('After Bilateral Filter', bilateral)
blur = cv2.GaussianBlur(bilateral, (5, 5), 0)
#cv2.imshow('After Gausian Blur', blur)

## Canny Edge Detection
edged = cv2.Canny(blur, 170, 200)
#cv2.imshow('After Canny Edge', edged)

## Find Contours
contours, hierarchy = cv2.findContours(edged, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key = cv2.contourArea, reverse = True)
# get list of rectangles contours
rectangle_contours = get_rect_list(contours)

## Hide plate
plate_contour = rectangle_contours[0]
# get the four points coord
rect = cv2.minAreaRect(plate_contour)
box = numpy.int0(cv2.boxPoints(rect))
# Fill the plate with WHITE color
final_image = cv2.drawContours(image_CV.copy(), [box], -1, (255, 255, 255), -1)
# Put the BORDER
final_image = cv2.drawContours(final_image, [box], 0, (207, 149, 1), 2)
# Put the plate
## resize the plate regarding the space of the contour
x,y,w,h = cv2.boundingRect(plate_contour)
#plate_image = image_CV[y:y+h, x:x+w]
#cv2.imshow('Plate', plate_image)
plate_CV_X = cv2.resize(plate_CV, (w,h))
final_image[y:y+h, x:x+w] = plate_CV_X


cv2.imshow('Original Image', image_CV)
cv2.imshow('Result', final_image)

cv2.waitKey(0)
cv2.destroyAllWindows()